#### Step 1: Prepare the data

import necessary libraries for loading the data:

nibabel for reading and writing neuroimaging data.
numpy for numerical operations and array handling.

In [1]:
import nibabel as nb
import numpy as np

This cell loads and preprocesses the brain imaging data and phenotype (demographic) information:

- Brain imaging data (cifti_data) is loaded from a .nii file.
- Network labels (network_label) are loaded and filtered to remove unspecified indices (-1).
- Binary vectors (net_6, net_7) are created for two specific networks, indicating vertex membership.
- Phenotype data (age, sex) is loaded and combined into a single array (phenotype).

In [2]:
# load brain imaging data
cifti_data = nb.load('./Dataset/HCP_WB_Tutorial_1.5_Pr_kN3mg/Q1-Q6_R440.All.sulc.32k_fs_LR.dscalar.nii').get_fdata(dtype=np.float32)

network_label = np.load('./Dataset/network_label_yeo7.npz')['arr_0']
idx = network_label!=-1

network_label = network_label[idx]
labeled_data = cifti_data[:,idx]
# generate binary vector for each network, 0-> specific vertex does not belong to the network, 1-> specific vertex belongs to the network.
net_6 = np.where(network_label!=6,0,1)
net_7 = np.where(network_label!=7,0,1)
# load phenotype data
age = np.load('./Dataset/R440_age.npz')['arr_0']
sex = np.load('./Dataset/R440_gender.npz')['arr_0']
phenotype = np.column_stack((sex, age))

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


Print the shapes of the processed data arrays to verify their dimensions, ensuring they are correct for subsequent analysis. In this example, the sample size is 440 and number of vertices is 58606.

In [3]:
labeled_data.shape, phenotype.shape, net_6.shape, net_7.shape

((440, 58606), (440, 2), (58606,), (58606,))

#### Step 2: Import the NEST package after the installation.

The package can be installed via 'pip install NEST'

In [4]:
from nest-sw import nest

#### Step 3: Define the following dictionary of arguments passed to the nest method. The args can be defined as follows, assuming vertex-wise linear models will be fit to estimate local brain-phenotype associations (i.e., specifying statFun='lm' in step 4.).

- X: N x P matrix (numpy array) of P imaging features (e.g., vertices) for N participants  (e.g., N=440, P=58606)

- y: N-dimensional vector of phenotype of interest (i.e., testing enrichment of X-y associations)

- Z: Optional. Specify one or more covariates (matrix with N rows and q columns for q covariates). Default is NULL (no covariates to be included).

- FL: Optional (default is False). Set to True to use Freedman-Lane procedure to account for dependence between covariates in permutation.

- n_perm: Optional (default is 999, with smallest possible p-value of 1/1000).

In [5]:
args = {
    'X': labeled_data,
    'y': phenotype[:,0],
    'Z': phenotype[:,1],
    'type': 'coef',
    'FL': False,
    'n_perm': 999
}

##### Step 4: Apply NEST to test enrichment of brain-phenotype associations in specified network.

In [6]:
pval,ES_obs,ES_null,_ = nest(statFun='lm',args=args,net_maps=net_6)

Using the following default settings: 


args['getNull'] = True ---> defaulting to get a null distribution



Print the p-value and observed enrichment score

In [9]:
pval, ES_obs

(0.394, {'net_0': 0.08281189583840687})

Print the enrichment scores for null distribution

In [10]:
ES_null

{'net_0': [0.13059220125447102,
  0.06471577073365589,
  0.09355988095385875,
  0.0818802092508586,
  0.12114088268632284,
  0.09799411720411677,
  0.04066773048527861,
  0.07303567374416853,
  0.09474520713954493,
  0.06511018816074765,
  0.06885833491826898,
  0.08719826354433158,
  0.07869418856847399,
  0.06844497746388506,
  0.05342036194677935,
  0.06515710684597398,
  0.11467653624530491,
  0.05909506792245156,
  0.08976863672081331,
  0.07559435739381881,
  0.0789532061654522,
  0.08691996299098526,
  0.1273247349165728,
  0.08208578162353208,
  0.1145382868378707,
  0.08025598555961588,
  0.08762862048290386,
  0.08373469620823565,
  0.06334170019494674,
  0.11173054142107897,
  0.06747348116528595,
  0.09180503122297112,
  0.020986410284107693,
  0.08846427604257764,
  0.0822432677191256,
  0.08102845779789458,
  0.06691629490183701,
  0.0472226292539355,
  0.0627381776813685,
  0.08472598169183854,
  0.0979176657934776,
  0.07599552181439484,
  0.08200492112477484,
  0.08066

NEST also supports the use of customized statistical functions for calculating associations. 
This advanced feature allows for more tailored analysis that fits specific research questions or datasets. 
To utilize a custom statistical function, specify the argument `statFun = 'your_custom_function'` 
alongside the corresponding arguments required by your function. 

In [ ]:
pval,ES_obs,ES_null,_ = nest(statFun='custom_fun',args=custom_args,net_maps=net_6)